In [ ]:
!pip install pyterrier
!pip install pyterrier[java]

  Using cached pyterrier-1.0.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached ir_datasets-0.5.11-py3-none-any.whl.metadata (12 kB)
  Using cached deprecated-1.3.1-py2.py3-none-any.whl.metadata (5.9 kB)
  Using cached ir_measures-0.4.3-py3-none-any.whl.metadata (7.0 kB)
  Using cached pytrec_eval_terrier-0.5.10-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (1.1 kB)
  Using cached lz4-4.4.5-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (3.8 kB)
  Using cached inscriptis-2.7.0-py3-none-any.whl.metadata (27 kB)
  Using cached ijson-3.4.0.post0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (23 kB)
  Using cached lxml-5.4.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.5 kB)
Using cached pyterrier-1.0.1-py3-none-any.whl (208 kB)
Using cached ir_datasets-0.5.11-py3-none-any.whl (866 kB)
Using cached ir_measures-0.4.3-py3-none-any.whl (61 kB)
Using cached lz4-4.4.5-cp312-cp312-

In [ ]:
import pyterrier as pt
import pickle
import pandas as pd
import shutil
import os
from pathlib import Path
import re

In [ ]:
dataset = pt.get_dataset('irds:beir/fiqa/test')

In [ ]:
for doc in dataset.get_corpus_iter():
    print(doc.keys())
    print(doc)
    break

[INFO] [starting] building docstore
[INFO] [starting] opening zip file
[INFO] If you have a local copy of https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/17918ed23cd04fb15047f73e6c3bd9d9
[INFO] [starting] https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip
docs_iter:   0%|                                     | 0/57638 [00:00<?, ?doc/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip: 0.0%| 0.00/17.9M [00:00<?, ?B/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip: 0.3%| 49.2k/17.9M [00:00<00:40, 439kB/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip: 1.0%| 180k/17.9M [00:00<00:21, 843kB/s] 
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip: 2.3%| 410k/17.9M [00:00<00:16, 1.05MB/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/dat

beir/fiqa/test documents:   0%|          | 0/57638 [00:00<?, ?it/s]

dict_keys(['text', 'docno'])
{'text': "I'm not saying I don't like the idea of on-the-job training too, but you can't expect the company to do that. Training workers is not their job - they're building software. Perhaps educational systems in the U.S. (or their students) should worry a little about getting marketable skills in exchange for their massive investment in education, rather than getting out with thousands in student debt and then complaining that they aren't qualified to do anything.", 'docno': '3'}


In [ ]:
dataset.get_corpus_iter()

beir/fiqa/test documents:   0%|          | 0/57638 [00:00<?, ?it/s]

In [ ]:
queries = dataset.get_topics()
display(queries)

[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]
[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]


,qid,query
0,4641,Where should I park my rainy-day / emergency f...
1,5503,Tax considerations for selling a property belo...
2,7803,Can the Delta be used to calculate the option ...
3,7017,Basic Algorithmic Trading Strategy
4,10152,What does a high operating margin but a small ...
...,...,...
643,4102,How can I determine if my rate of return is “g...
644,3566,Where can I buy stocks if I only want to inves...
645,94,Using credit card points to pay for tax deduct...
646,2551,How to find cheaper alternatives to a traditio...


In [ ]:
qrels = dataset.get_qrels()
display(qrels)

,qid,docno,label,iteration
0,8,566392,1,0
1,8,65404,1,0
2,15,325273,1,0
3,18,88124,1,0
4,26,285255,1,0
...,...,...,...,...
1701,11039,330058,1,0
1702,11039,91183,1,0
1703,11054,155053,1,0
1704,11054,321015,1,0


In [ ]:
index_path =  str(Path("./beir-index").resolve())

In [ ]:
os.path.exists(index_path)

False

In [ ]:
def ensure_java_home():
    if os.environ.get("JAVA_HOME"):
        return
    java = shutil.which("java.exe" if os.name == "nt" else "java")
    if java:
        jdk_parent = Path(java).resolve().parents[1]
        cand = jdk_parent / ("bin/java.exe" if os.name == "nt" else "bin/java")
        if cand.exists():
            os.environ["JAVA_HOME"] = str(jdk_parent)
            return
    if not os.environ.get("JAVA_HOME"):
        raise EnvironmentError(
            "JAVA_HOME is not set. Install a JDK and set JAVA_HOME to the JDK folder.\n"
        )

ensure_java_home()

In [ ]:
if os.path.exists(index_path):
    shutil.rmtree(index_path)
indexer= pt.index.IterDictIndexer(index_path, fields= True, meta={'docno':15}, text_attrs=['text'])

indexref = indexer.index(dataset.get_corpus_iter())

terrier-assemblies 5.11 jar-with-dependencies not found, downloading to /root/.pyterrier...


https://repo1.maven.org/maven2/org/terrier/terrier-assemblies/5.11/terrier-assemblies-5.11-jar-with-dependenci…

Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...


https://repo1.maven.org/maven2/org/terrier/terrier-python-helper/0.0.8/terrier-python-helper-0.0.8.jar:   0%| …

Done


Java started (triggered by TerrierIndexer.__init__) and loaded: pyterrier.java.colab, pyterrier.java, pyterrier.java.24, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


beir/fiqa/test documents:   0%|          | 0/57638 [00:00<?, ?it/s]

23:44:38.607 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (7915) - further warnings are suppressed
23:45:03.579 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer -- Indexed 39 empty documents


In [ ]:
index = pt.IndexFactory.of(indexref)
stats = index.getCollectionStatistics()

In [ ]:
print("Index folder:", index_path)
print("Number of documents:", stats.getNumberOfDocuments())
print("Number of postings:", stats.getNumberOfPostings())
print("Number of tokens:", stats.getNumberOfTokens())
print("Number of unique terms:", stats.getNumberOfUniqueTerms())
print(f"Average document length:{stats.getAverageDocumentLength():.2f}")


Index folder: /content/beir-index
Number of documents: 57638
Number of postings: 2714611
Number of tokens: 3783214
Number of unique terms: 51260
Average document length:65.64


In [ ]:
q_text = dataset.get_topics("text")
q_text

,qid,query
0,4641,Where should I park my rainy-day / emergency f...
1,5503,Tax considerations for selling a property belo...
2,7803,Can the Delta be used to calculate the option ...
3,7017,Basic Algorithmic Trading Strategy
4,10152,What does a high operating margin but a small ...
...,...,...
643,4102,How can I determine if my rate of return is “g...
644,3566,Where can I buy stocks if I only want to inves...
645,94,Using credit card points to pay for tax deduct...
646,2551,How to find cheaper alternatives to a traditio...


In [ ]:
counts = qrels.groupby("qid")["docno"].count()  # group by query id and count documents
print("Overall Statistics")
print(counts.describe())

Overall Statistics
count    648.000000
mean       2.632716
std        2.062616
min        1.000000
25%        1.000000
50%        2.000000
75%        3.000000
max       15.000000
Name: docno, dtype: float64


In [ ]:
bm25 = pt.terrier.Retriever(indexref, wmodel='BM25', controls={"qtype": "matchop"})
tfidf = pt.terrier.Retriever(indexref, wmodel="TF_IDF", controls={"qtype": "matchop"})

In [ ]:
def rmv_problems(text: str) -> str:
    text = str(text)

    text = text.replace(":", " ")
    text = text.replace("'", " ")
    text = text.replace('"', " ")
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [ ]:
q_text['query'] = q_text['query'].apply(rmv_problems)

In [ ]:
res_bm25 = bm25.transform(q_text[['qid','query']])
res_tfidf = tfidf.transform(q_text[['qid','query']])

In [ ]:
res_bm25

,qid,docid,docno,rank,score,query
0,4641,36224,376148,0,41.677305,Where should I park my rainy-day / emergency f...
1,4641,47916,497993,1,29.149791,Where should I park my rainy-day / emergency f...
2,4641,55690,580025,2,26.773005,Where should I park my rainy-day / emergency f...
3,4641,24501,253614,3,26.640181,Where should I park my rainy-day / emergency f...
4,4641,3157,32833,4,24.265187,Where should I park my rainy-day / emergency f...
...,...,...,...,...,...,...
645148,2399,25037,259223,995,10.230066,Where do web sites get foreign exchange curren...
645149,2399,33542,348029,996,10.229760,Where do web sites get foreign exchange curren...
645150,2399,13368,137444,997,10.221526,Where do web sites get foreign exchange curren...
645151,2399,22944,237573,998,10.219489,Where do web sites get foreign exchange curren...


In [ ]:
from pyterrier.measures import *

bm25_evaluated = pt.Evaluate(res_bm25, dataset.get_qrels(), metrics=[AP@10,P@5,P@10], perquery=False)
tfidf_evaluated = pt.Evaluate(res_tfidf, dataset.get_qrels(), metrics=[AP@10,P@5,P@10], perquery=False)


In [ ]:
bm25_evaluated

{'AP@10': 0.19396587696466464,
 'P@5': 0.10648148148148186,
 'P@10': 0.07037037037037075}

In [ ]:
tfidf_evaluated

{'AP@10': 0.19392245231599844,
 'P@5': 0.10864197530864238,
 'P@10': 0.07114197530864234}

In [ ]:
print(shutil.which("perl"))

/usr/bin/perl


In [ ]:
os.environ["PATH"] = r"C:\Strawberry\perl\bin;" + os.environ["PATH"]

In [ ]:
import re
topics = dataset.get_topics("text").copy()

def clean_q(q: str) -> str:
    q = q.replace(":", " ")
    q = q.replace('"', " ")
    q = re.sub(r"\s+", " ", q).strip()
    return q

topics["query"] = topics["query"].map(clean_q)

In [ ]:
pt.Experiment(
    [tfidf, bm25],
    topics,
    dataset.get_qrels(),
    eval_metrics=["P_10", P@10],
    names=["TF_IDF", "BM25"],
    baseline=0,
    verbose=True,
)

pt.Experiment:   0%|          | 0/2 [00:00<?, ?system/s]

,name,P_10,P_10 +,P_10 -,P_10 p-value
0,TF_IDF,0.071142,NaN,NaN,NaN
1,BM25,0.070370,5.0,11.0,0.251657


In [ ]:
cnt = 0
for doc in dataset.get_corpus_iter():
    print(doc.keys())
    print(doc)
    cnt += 1
    if cnt == 5:
      break

beir/fiqa/test documents:   0%|          | 0/57638 [00:00<?, ?it/s]

dict_keys(['text', 'docno'])
{'text': "I'm not saying I don't like the idea of on-the-job training too, but you can't expect the company to do that. Training workers is not their job - they're building software. Perhaps educational systems in the U.S. (or their students) should worry a little about getting marketable skills in exchange for their massive investment in education, rather than getting out with thousands in student debt and then complaining that they aren't qualified to do anything.", 'docno': '3'}
dict_keys(['text', 'docno'])
{'text': "So nothing preventing false ratings besides additional scrutiny from the market/investors, but there are some newer controls in place to prevent institutions from using them. Under the DFA banks can no longer solely rely on credit ratings as due diligence to buy a financial instrument, so that's a plus. The intent being that if financial institutions do their own leg work then *maybe* they'll figure out that a certain CDO is garbage or not. 